
# qLearn Week 4: Multi-Qubit Quantum Gates & Entanglement

This notebook provides challenges based on quantum computing concepts we learned this week: quantum entanglement, multi-qubit gates, quantum oracles, and Grover's Algorithm. 

### Topics:
1. Quantum Entanglement
2. Multi-Qubit Gates
3. Quantum Oracles
4. Grover's Algorithm

### Requirements
Ensure you have installed PennyLane: `pip install pennylane`.
Pennylane documentation can be found [here](https://docs.pennylane.ai/en/stable/index.html).


In [7]:
# Installing and Importing Necessary Libraries

# Install PennyLane if it's not already installed
# Uncomment the line below to install PennyLane in the Jupyter environment
#%pip install pennylane --upgrade

# Importing PennyLane library to create and execute quantum circuits
import pennylane as qml
dev = qml.device('default.qubit',wires=1)
# Import NumPy from PennyLane, which is required for some quantum calculations
from pennylane import numpy as np

# Import visualization for plotting Bloch spheres if desired (using matplotlib here)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define a device with 2 qubits
dev = qml.device("default.qubit", wires=2)

In [45]:
import matplotlib.pyplot as plt
secret_target_pattern = [1, 1]
def display_probabilities(probabilities, labels=None):
    
    percentages = [100*np.abs(prob)**2 for prob in probabilities]
    num_states = len(probabilities)
    if labels is None:
        labels = [f"|{i:0{num_states.bit_length() - 1}b}⟩" for i in range(num_states)]

    plt.figure(figsize=(8, 5))
    plt.bar(labels, percentages, color='royalblue', alpha=0.7)
    plt.xlabel("Quantum States")
    plt.ylabel("Probability (%)")
    plt.title("State Probabilities")
    plt.ylim(0, 100)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()



## Challenge 1: Quantum Entanglement
### Background:
The Bell state is a fundamental example of quantum entanglement, representing a pair of qubits in a maximally entangled state. It is one of the simplest and most studied entangled states, often expressed as: 
$$
|\Phi^+\rangle = \frac{1}{\sqrt{2}} \big( |00\rangle + |11\rangle \big)
$$

### Task:
1. Create a Bell State using a quantum circuit.
2. Measure the state to verify entanglement by checking correlations.


### Code:


In [ ]:
@qml.qnode(dev)
def bell_state():
    #######################
    #   YOUR CODE HERE    #
    #######################

    return qml.state()

# Generate the Bell state
state = bell_state()
print("Bell State:", state)

display_probabilities(state)


## Challenge 2: Multi-Qubit Gates

### Task:
1. Implement the SWAP gate using its matrix representation.
2. Confirm the operation by applying it to a two-qubit state.

### Hint:
The SWAP gate swaps the states of two qubits. To apply a quantum gate in terms of its unitary, you can use the QubitUnitary function in Pennylane. More about the QubitUnitary function can be found [here](https://docs.pennylane.ai/en/stable/code/api/pennylane.QubitUnitary.html).<br />
REMEMBER: Qubits start in state 0, apply X gates to see outputs for other possible input states.
### Code:


In [ ]:

@qml.qnode(dev)
def apply_swap():
    #######################
    #   YOUR CODE HERE    #
    #######################
    # Define the SWAP gate matrix
    
    # Apply the custom unitary

    return qml.state()

# Check the effect of the SWAP gate
print("State after SWAP gate:", apply_swap())

display_probabilities(apply_swap())



## Challenge 3: Quantum Oracles
In lecture, we had discussed the idea of finding the combination to a lock using an oracle. Applying the oracle to a correct combination will flip its sign, so we can use it to determine the (unknown) combination of the lock.<br />
Reminder that we can represent the oracle as a unitary operator: 
$$
U_{f} = I - 2 |s\rangle \langle s| 
$$
### Task:
1. In <code>oracle_matrix(targetstate)</code>, we want to flip the sign of the index corresponding to the solution state <code>targetstate</code>, so change the index of the matrix at that point to -1.
2. Now in <code>oracle_circuit(combo)</code>, implement the Oracle circuit discussed in class (ie. apply uniform superposition to every qubit, then apply the oracle unitary). <code>combo</code> is a secret input state of whose phase should be flipped.

### Hint:
When creating the circuit, make sure to understand each level of the circuit. What creates uniform superposition? How would we apply a unitary? <br />
Remember, the oracle changing the phase of a state DOES NOT affect its probability, so you will have to observe the state itself to figure out whats going on!
### Code:


In [ ]:
def oracle_matrix(targetstate):
    index = np.ravel_multi_index(targetstate, [2]*len(targetstate)) # Index of solution
    my_array = np.identity(2**len(targetstate)) # Identity matrix

    ##################
    # YOUR CODE HERE #
    ##################
    # MODIFY DIAGONAL ENTRY CORRESPONDING TO SOLUTION INDEX

    return my_array


@qml.qnode(dev)
def oracle_circuit(combo):
    ##################
    # YOUR CODE HERE #
    ##################

    #Create Uniform Superposition across all (2 in this case) qubits

    # Apply the oracle: Flip the phase of the marked state
    

    return qml.state()
print(oracle_circuit(secret_target_pattern))
display_probabilities(oracle_circuit(secret_target_pattern))


## Challenge 4: Grover's Algorithm
Grover's Algorithm can be represented using the circuit below, where the first n qubits (<code>query_register</code>) represent the qubits part of the search, while the last (nth) qubit, (<code>aux</code>), represents an additional qubit needed to perform the transformations. For this challenge, we will focus on HOW Grover's Algorithm has an advantage over classical searches.

![Grover's Circuit](https://assets.cloud.pennylane.ai/codebook/grover-iter-2.svg)
### Task:
Challenge 1. Implement Grover's Algorithm to find a marked state in a n-qubit system in n steps (<code>num_steps</code>) <br />
Challenge 2. For an increasing number of qubits, determine the optimal number of steps for a Grover Search

### Code:


In [ ]:
def hadamard_transform(my_wires):
    for wire in my_wires:
        qml.Hadamard(wires=wire)

def oracle(combo,n_bits):
    query_register = list(range(n_bits))
    aux = [n_bits]
    combo_str = "".join(str(j) for j in combo)
    qml.MultiControlledX(query_register,aux,combo_str)
    pass

def diffusion(n_bits):
    query_register = list(range(n_bits))
    aux = [n_bits]
    qml.X(aux)
    hadamard_transform(query_register)
    for wire in range(len(query_register)):
        qml.X(wire)
    qml.MultiControlledX(query_register,aux)
    for wire in range(len(query_register)):
        qml.X(wire)
    hadamard_transform(query_register)
    pass


def grover_search(combo, num_steps):
    n_bits = len(combo)
    query_register = list(range(n_bits))
    aux = [n_bits]
    all_wires = query_register + aux
    dev = qml.device("default.qubit", wires=all_wires)

    @qml.qnode(dev)
    def inner_circuit():
        ##################
        # YOUR CODE HERE #
        ##################
        # IMPLEMENT THE GROVER CIRCUIT
        

    return inner_circuit()

In [ ]:
n_list = range(3, 7)
opt_steps = []

def local_max_arg(num_list):
    for i in range(1,len(num_list) - 1):
        if num_list[i] > num_list[i-1] and num_list[i] > num_list[i+1]:
            return i+1
    return 1



for n_bits in n_list:
    combo = "0" * n_bits  # A simple combination
    step_list = range(1, 10)  # Try out some large number of steps
    ##################
    # YOUR CODE HERE #
    ##################
     
    pass

print("The optimal number of Grover steps for qubits in", [3, 4, 5, 6], "is", opt_steps, ".")
